In [1]:
from transformer_lens import HookedTransformer
from mechinterp.Interpreter import Interpreter

In [7]:
model = HookedTransformer.from_pretrained("gpt2")
interp = Interpreter(model)

Loaded pretrained model gpt2 into HookedTransformer


In [8]:
logits, cache = model.run_with_cache("It was the best of times")

In [ ]:
# Interpreting hidden activations
interp.tuned_lens(cache["blocks.10.hook_resid_post"][0, 5], l=10, k=4)

Tuned Logit Lens Output:
	- Topk tokens: [' times', ' worlds', ' intentions', ' highs']

	- Bottomk tokens: ['anwhile', 'hyde', 'ividual', 'StreamerBot']

In [16]:
interp.tuned_lens(cache["blocks.7.hook_resid_post"][0, 3], l=7, k=4)

Tuned Logit Lens Output:
	- Topk tokens: [' dawn', ' culmination', ' hottest', ' last']

	- Bottomk tokens: ['omever', 'ambo', 'apons', 'swer']